# IMPORTANTE

La Tarea 01 de Fundamento de Procesamiento de Imágenes debe realizarse siguiendo este esqueleto.

# 1. LECTURA DE IMAGENES
(no modificar)

In [ ]:
!wget https://www.dropbox.com/scl/fi/ew41atgii3kvwcbp3so1m/Tarea01Imagenes.zip?rlkey=hlwxpu7ll8g5na2tv9ajwi81f
!mv Tarea01Imagenes.zip?rlkey=hlwxpu7ll8g5na2tv9ajwi81f Tarea01Imagenes.zip
!unzip -qq Tarea01Imagenes.zip

# 2. LIBRERIAS PERMITIDAS
(no modificar)

In [206]:
import matplotlib.pyplot as plt
import numpy as np

# 3. FUNCIONES NECESARIAS
(no modificar)

In [207]:
def num2fixstr(x,d):
    st = '%0*d' % (d,x)
    return st

def time2str(ti):
  st = num2fixstr(ti[0],2)+':'+num2fixstr(ti[1],2)+':'+num2fixstr(ti[2],2)
  return st

# 4. SOLUCION PROPUESTA

In [208]:
def plot_all(image_array, gray=False):
    plt.figure().set_figwidth(30)
    plt.grid(False)
    plt.axis('off')
    if gray:
        plt.imshow(np.concatenate(image_array,axis=1), cmap='gray')
    else:
        plt.imshow(np.concatenate(image_array,axis=1))
    plt.show()

In [209]:
def blur(image, matrix_n):
    """
    Recibe una imagen de Y x X x 1 y un entero N, retorna la matriz difuminada por un filtro de promedio de NxN.
    """

    N = int((matrix_n-1)/2)
    image_ = np.zeros(image.shape)

    for n in range(N):
        image[n, :] = 0
        image[-n, :] = 0
        image[:, n] = 0
        image[:, -n] = 0

    for x in range(-N, N+1):
        for y in range(-N, N+1):
            image_[:,:] += np.roll(image, (x, y), (0, 1))/matrix_n**2

    return image_

In [210]:
def filterr(image, limit=128):
    """
    Recibe una imagen y un límite, y devuelve una imagen solo con los pixeles que superan el límite reescalados a 255.
    """
    return image * (image > limit) / (limit/255)

In [211]:
def umbral(image, umbral):
    """
    Recibe una imagen y un umbral y devuelve una imagen binaria
    """
    return np.ones(image.shape) * (image > umbral) * 255

In [212]:
def umbral_range(image, a, b):
    """
    Recibe una imagen y dos valores a y b, y devuelve una imagen binaria con valor 255 en los pixeles que estan entre a y b, y 0 en los demas.
    """
    return np.ones(image.shape) * (a < image[:,:]) * (image[:,:] < b) * 255

In [213]:
def column_filter(image, n):
    """
    Filtra las columnas de la imagen, dejando en 0 las que tengan menos de n pixeles blancos
    """
    for i in range(image.shape[0]):
        if np.sum(image[i,:])/255 < n:
            image[i,:] = 0
    return image

def fila_filter(image, n):
    """
    Filtra las filas de la imagen, dejando en 0 las que tengan menos de n pixeles blancos
    """
    for i in range(image.shape[1]):
        if np.sum(image[:,i])/255 < n:
            image[:,i] = 0
    return image

In [214]:
def get_limit(image, size=None):
	"""
	Recibe una imagen binaria y devuelve los limites de la imagen, si se recibe un tamaño, devuelve los limites de la imagen escalados a ese tamaño.
	"""
	top = -1
	for n in range(image.shape[0]):
		if np.sum(image[n,:]) > 0 and top == -1:
			top = n - 12

	bottom = -1
	for n in range(image.shape[0]-1, 0, -1):
		if np.sum(image[n,:]) > 0 and bottom == -1:
			bottom = n

	left = -1
	for n in range(image.shape[1]):
		if np.sum(image[:,n]) > 0 and left == -1:
			left = n - 12

	right = -1
	for n in range(image.shape[1]-1, 0, -1):
		if np.sum(image[:,n]) > 0 and right == -1:
			right = n

	if size != None:
		top = int((top + 12)*size[1]/image.shape[1])
		bottom = int((bottom )*size[1]/image.shape[1])
		left = int((left + 12)*size[0]/image.shape[0])
		right = int((right )*size[0]/image.shape[0])

		return [top, bottom, left, right]

	return [top, bottom, left, right]

In [215]:
def get_inner_limit(image):
    """
    Recibe una imagen binaria y devuelve los limites de la imagen evaluados en un rango 'm' desde el centro de la imagen.
    """

    v_range = image.shape[0]/2
    h_range = image.shape[1]/2




    tolerancia = 30

    cantidad = 5

    mass = 40

    left = -1
    for n in range(image.shape[1]):
        dat = (image[int(v_range - tolerancia):int(v_range + tolerancia + 1), n])
        if np.sum(dat)/255 > cantidad and left == -1:

            L = []
            for w in range(mass):
                dat = (image[int(v_range - tolerancia):int(v_range + tolerancia + 1), n + w])
                for nn in range(dat.shape[0]):
                    if dat[nn] > 0:
                        L.append(nn - tolerancia)
            L = sum(L)/len(L)

            left = n

    right = -1
    for n in range(image.shape[1]-1, 0, -1):
        dat = (image[int(v_range - tolerancia):int(v_range + tolerancia + 1), n])
        if np.sum(dat)/255 > cantidad and right == -1:

            R = []
            for w in range(mass):
                dat = (image[int(v_range - tolerancia):int(v_range + tolerancia + 1), n + w])
                for nn in range(dat.shape[0]):
                    if dat[nn] > 0:
                        R.append(nn - tolerancia)
            R = sum(R)/len(R)

            right = n


    H = max(R, L) - min(R, L)

    angle = np.arctan2(H, right - left - mass)*180/np.pi

    m = 10


    top = -1
    for n in range(image.shape[0]):
        if np.sum(image[n,int(h_range - m):int(h_range + m)]) > 0 and top == -1:
            top = n




    bottom = top + (right - left)
    return [top, bottom, left, right, angle]

In [216]:
def get_hh_mm(image, aa=0):
    """
    Recibe un imagen binaria de un reloj y devuelve la hora y los minutos
    """

    x_base = image.shape[1]/2
    y_base = image.shape[0]/2

    lista_1 =[]
    lista_2 =[]
    lista_3 =[]

    p1 = 0.2
    p2 = 0.39
    p3 = 0.49

    """Revisa los pixeles de la imagen en 3 radios distintos, y guarda los angulos en los que hay pixeles blancos"""
    for n in range(0, 360, 1):
        n = -(n - 90)
        x1 = int(x_base * (1 + np.cos(n*np.pi/180)*p1))
        y1 = int(y_base * (1 - np.sin(n*np.pi/180)*p1))

        x2 = int(x_base * (1 + np.cos(n*np.pi/180)*p2))
        y2 = int(y_base * (1 - np.sin(n*np.pi/180)*p2))

        x3 = int(x_base * (1 + np.cos(n*np.pi/180)*p3))
        y3 = int(y_base * (1 - np.sin(n*np.pi/180)*p3))


        if image[y1, x1] > 0:
            lista_1.append(n)
        if image[y2, x2] > 0:
            lista_2.append(n)
        if image[y3, x3] > 0:
            lista_3.append(n)


    listas = [lista_1, lista_2, lista_3]

    """Revisa los angulos que estan muy cerca y los junta"""
    for i in range(3):
        n = 0
        m = 1
        while n < len(listas[i])-1:
            if abs(listas[i][n]/m - listas[i][n+1]) < 10:
                listas[i][n] += listas[i].pop(n+1)
                m += 1
            else:
                listas[i][n] /= m
                m = 1
                n += 1
        listas[i][-1] /= m


    """Asigna los angulos a las agujas"""
    for i in range(len(listas[0])):
        for j in range(len(listas[1])):
            for k in range(len(listas[2])):
                if (listas[0][i] - 10 < listas[1][j] < listas[0][i] + 10) and (listas[0][i] - 10 < listas[2][k] < listas[0][i] + 10):
                    a = [listas[0][i], listas[1][j], listas[2][k]]

    for i in range(len(listas[0])):
        for j in range(len(listas[1])):
            for k in range(len(listas[2])):
                if (listas[0][i] - 10 < listas[1][j] < listas[0][i] + 10):
                    if listas[0][i] not in a and listas[1][j] not in a:
                        b = [listas[0][i], listas[1][j]]


    ang_a = -sum(a)/len(a) + 90 + aa
    ang_b = -sum(b)/len(b) + 90 + aa



    minu = ang_a*60/360
    hora = ang_b*12/360

    return [int(hora), int(minu)]


In [217]:
def get_ss(image, a=0):
    """
    Recibe un imagen binaria de un reloj y devuelve los segundos
    """
    x_base = image.shape[1]/2
    y_base = image.shape[0]/2

    """Revisa los pixeles de la imagen en 3 radios distintos, y guarda los angulos en los que hay pixeles blancos"""
    dict_data = {"0.2": [], "0.3": [], "0.4":[], "0.6":[]}
    for n in range(0, 3600, 1):
        n = -(n/10 - 90)

        for d in dict_data:
            x = int(x_base * (1 + np.cos(n*np.pi/180)*float(d)))
            y = int(y_base * (1 - np.sin(n*np.pi/180)*float(d)))
            if image[y, x] > 0:
                dict_data[d].append(n)


    listas = []

    ss = 0

    """Revisa los angulos que estan muy cerca y los junta"""
    for i in dict_data:
        n = 0
        m = 1
        while n < len(dict_data[i])-1:
            if abs(dict_data[i][n]/m - dict_data[i][n+1]) < 10:
                dict_data[i][n] += dict_data[i].pop(n+1)
                m += 1
            else:
                dict_data[i][n] /= m
                m = 1
                n += 1
        dict_data[i][-1] /= m


        listas.append(dict_data[i])

    """Busca angulos que esten en los 4 radios y calcula los segundos"""
    ss = 0
    for i in range(len(listas[0])):
        for j in range(len(listas[1])):
            for k in range(len(listas[2])):
                for t in range(len(listas[3])):
                    if (listas[0][i] - 6 < listas[1][j] < listas[0][i] + 6) and \
                        (listas[1][j] - 6 < listas[2][k] < listas[1][j] + 6) and \
                        (listas[2][k] - 6 < listas[3][t] < listas[2][k] + 6):
                        ss = sum([listas[0][i], listas[1][j], listas[2][k], listas[3][t]])/4


    return round((-(ss) + a*2 + 90)*60/360 + 0.5)



In [218]:
def QueHoraEs(imgpath, show=False):
    N = 12
    mask = plt.imread(imgpath)[::N, ::N]


    mask = filterr(mask[:,:,0], 165)
    mask = blur(mask, 5)
    mask = umbral(mask, 235)
    mask = fila_filter(column_filter(mask, 15), 15)

    N = 2
    top, bottom, left, right = get_limit(mask, [1512, 2016])
    image = plt.imread(imgpath)[::N, ::N][top:bottom, left:right, :]


    s = int(image.shape[0]/400)
    new = image[::s, ::s, :]


    R1 = umbral_range(new[:, :, 0], 20, 120)
    # G1 = umbral_range(new[:, :, 1], 20, 120)
    # B1 = umbral_range(new[:, :, 2], 20, 120)
    total_hm = R1

    top, bottom, left, right, ajust = get_inner_limit(total_hm)
    # total_hm = total_hm[top:bottom, left:right]

    hh, mm = get_hh_mm(total_hm, ajust)



    R2 = umbral_range(new[:,:, 0], 140, 205)
    G2 = umbral_range(new[:,:, 1], 145, 200)
    # B2 = umbral_range(new[:,:, 1], 145, 200)
    total_seg = R2*G2

    total_seg = total_seg[top:bottom, left:right]
    ss = get_ss(total_seg)



    if show:
        plt.imshow(image)
        plt.show()


    tt = np.zeros(3,)
    tt[0] = hh
    tt[1] = mm
    tt[2] = ss

    return tt



# 5. PRUEBA DE ALGORITMO PROPUESTO
(opcional)

In [219]:
t3 = QueHoraEs('clock_05.jpg',show=0)
print('Son las '+time2str(t3))

Son las 11:11:05


# 6. EVALUACION DE PRECISION
(no modificar)

Aquí se calcula A, la precisión de la tarea que es un número entre 0 y 1.

**Entre más cercano es A al valor  1.000, mejor es la precisión del algoritmo propuesto.**

A se define como (A1+A2+...+A5+A6)/6, donde Ai, la precision en la imagen i, se calcula como:

(max(0,(10-Si)/10)+max(0,(10-Mi)/10)+max(0,(3-Hi)/3)/3

con Si,Mi,Hi es el error en segundos, minutos y horas respectivamente en la estimación de la Hora.

In [220]:
# Ground Truth: Hora Real
GT = [2 ,44,23,   # hora de clock_01.jpg
      3 ,37, 5,   # hora de clock_02.jpg
      5 ,21,48,   # hora de clock_03.jpg
      8 ,21,27,   # hora de clock_04.jpg
      11,11, 4,   # hora de clock_05.jpg
      1 ,56,15]   # hora de clock_06.jpg

In [221]:
n = 6
TT = np.zeros((n,3))
AA = np.zeros(n,)
print('Imagen         Real      Estimado  Error     Precisión')
print('------------------------------------------------------')
for i in range(n):
  k       = i+1
  fimg    = 'clock_'+num2fixstr(k,2)+'.jpg'
  ti      = QueHoraEs(fimg)
  TT[i,:] = ti
  gi      = GT[i*3:i*3+3]
  ei      = np.abs(ti-gi)
  Si      = ei[2]
  Mi      = ei[1]
  Hi      = ei[0]
  ai      = (np.max([0,(10-Si)/10])+np.max([0,(10-Mi)/10])+np.max([0,(3-Hi)/3]))/3
  AA[i]   = ai
  print(fimg       +' : '+time2str(gi)+'  '+time2str(ti)+'  '+time2str(ei)+'  '+f'{ai:.4f}')
print('------------------------------------------------------')

A = np.mean(AA)

print('                   Precision de la Tarea A = '+f'{A:.4f}')
print('------------------------------------------------------')



Imagen         Real      Estimado  Error     Precisión
------------------------------------------------------
clock_01.jpg : 02:44:23  02:44:23  00:00:00  1.0000
clock_02.jpg : 03:37:05  03:37:05  00:00:00  1.0000
clock_03.jpg : 05:21:48  05:22:48  00:01:00  0.9667
clock_04.jpg : 08:21:27  08:20:27  00:01:00  0.9667
clock_05.jpg : 11:11:04  11:11:05  00:00:01  0.9667
clock_06.jpg : 01:56:15  01:56:15  00:00:00  1.0000
------------------------------------------------------
                   Precision de la Tarea A = 0.9833
------------------------------------------------------
